In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 3)
plt.rcParams["axes.grid"] = True

df = pd.read_csv("../csv/global_returns.csv", delimiter=';')

# Global log

In [ ]:
df.shape

In [ ]:
df.head(20)

In [ ]:
df.dtypes

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.dtypes

In [ ]:
df.query('"QueueConsumer.run()" == name')

In [ ]:
df.query('"QueueConsumer.run()" != name').sort_values(by='elapsed', ascending=False).head(50)

In [ ]:
df["module"].value_counts().plot(kind="bar", title="Function calls by module")
plt.show()

In [ ]:
df['thread'].value_counts().plot(kind="pie", title="Function calls by thread")
plt.show()

In [ ]:
df['timestamp']

In [ ]:
df.plot(kind='scatter',
       x='timestamp',
       y='module')

In [ ]:
df.plot(kind="scatter", x='timestamp', y='thread')

In [ ]:
df.plot(kind="scatter", x='thread', y='module')

In [ ]:
df['name'].value_counts()